***Determining Optimimal Number of Neurons in Hidden Layer of ANN***
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [27]:
# Step 1: Load the data
data=pd.read_csv('./resources/data/Churn_Modelling.csv')

# Step 2: Preprocess the data
# Step 2.1 Remove unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Step 2.2 Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

# Step 2.3: One-hot encode 'Geography' column
onehot_encoder_geography = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

# Step 3 - Split the data into training and testing sets
# x = inputs = all columns except 'Exited'
# y = output = 'Exited' column
x = data.drop('Exited', axis=1) 
y = data['Exited']

# Step 4 - Split the data in to training and testing sets
x_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Step 5 - Scale the data
scaler_xtrain_x_test = StandardScaler()
x_train = scaler_xtrain_x_test.fit_transform(x_train)
x_test = scaler_xtrain_x_test.transform(X_test)

# Step 6 - Save encoders and scaler for later use
with open('./resources/pickle/3_label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('./resources/pickle/3_onehot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geography, file)

with open('./resources/pickle/3_scaler_xtrain_x_test.pkl', 'wb') as file:
    pickle.dump(scaler_xtrain_x_test, file)

In [ ]:
# Step 7 - Find out whats the best parameters for our ANN model using GridSearchCV

# Step 7.1 - Create a function to build an ANN model based on the parameters passed(KerasClassifier)
# if nuerons and layers are not passed, default values (neurons=32, layers=1) will be used 
def create_ann_model(neurons=32,hidden_layers=1):
    ann_model=Sequential()
    ann_model.add(Dense(neurons,activation='relu',input_shape=(x_train.shape[1],))) # One Input layer, One Hidden Layer

    # Create hidden layers
    # We are doing hidden_layers-1 because one hidden layer is already created above
    for _ in range(hidden_layers-1):
        ann_model.add(Dense(neurons,activation='relu'))

    # Output Layer
    # - If we only did this for the output layer = ann_model(Dense(1)), i.e. did not provide an activation function, then the default activation function = 'Linear' which is used for Regression problems.
    ann_model.add(Dense(1,activation='sigmoid'))
    ann_model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return ann_model

In [ ]:
# Step 7.2 - Create KerasClassifier
# - KerasClassifier is responsible for creating the entire ANN model here using the configuration in the function - create_ann_model()
keras_classifier = KerasClassifier(model=create_ann_model, verbose=1)

# Step 7.3 - Define the parameter grid to search
# These are the options we are providing to GridSearchCV to find the best combination of Neurons, Layers and Epochs
grid_parameters = {
    'model__neurons': [16, 32, 64, 128],
    'model__hidden_layers': [1, 2],
    'epochs': [50, 100]
}

In [ ]:
# Step 7.4) Perform Grid Search with Cross Validation
'''
GridSearchCV will take each combination provided by 'grid_parameters' and find out what is the most optimal combination of Neurons, Layers and Epochs.

cv=3 means 3-Fold Cross Validation
 - Your dataset is split into 3 equal parts (folds).
 - The model is trained 3 times for each hyperparameter combination:
 - Each time, 2 folds are used for training
 - 1 fold is used for validation

The accuracy scores from the 3 runs are averaged.
GridSearchCV uses this average score to select the best hyperparameters.

Example:
Fold 1 → Train on (2,3), Validate on 1
Fold 2 → Train on (1,3), Validate on 2
Fold 3 → Train on (1,2), Validate on 3
'''

grid_search = GridSearchCV(estimator=keras_classifier, param_grid=grid_parameters, scoring='accuracy', cv=3)
grid_result = grid_search.fit(x_train, y_train)

# Print the best parameters
# Output observation : Best: 0.856000 using {'epochs': 50, 'model__hidden_layers': 1, 'model__neurons': 128}
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/50


/Users/ankur/backup/delta/sn/datascience/workspace/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4952 - loss: 0.7483
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7905 - loss: 0.5122
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8039 - loss: 0.4576
Epoch 4/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8119 - loss: 0.4368
Epoch 5/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8183 - loss: 0.4250
Epoch 6/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8221 - loss: 0.4163
Epoch 7/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8247 - loss: 0.4076
Epoch 8/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8296 - loss: 0.3992
Epoch 9/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8367 - loss: 0.3915
Epoch 10/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8393 - loss: 0.3840
Epoch 11/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8414 - loss: 0.3779
Epoch 12/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy

/Users/ankur/backup/delta/sn/datascience/workspace/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6128 - loss: 0.6628
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7928 - loss: 0.4861
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8039 - loss: 0.4526
Epoch 4/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8091 - loss: 0.4402
Epoch 5/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8127 - loss: 0.4331
Epoch 6/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8174 - loss: 0.4266
Epoch 7/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8232 - loss: 0.4196
Epoch 8/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8249 - loss: 0.4113
Epoch 9/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8311 - loss: 0.4031
Epoch 10/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8369 - loss: 0.3951
Epoch 11/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8399 - loss: 0.3883
Epoch 12/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy

...

167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8211 - loss: 0.4336
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8502 - loss: 0.3702
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8509 - loss: 0.3522
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8586 - loss: 0.3446
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8603 - loss: 0.3382
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8614 - loss: 0.3346
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8650 - loss: 0.3285
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8667 - loss: 0.3275
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8657 - loss: 0.3234
Epoch 10/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.3197
Epoch 11/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8663 - loss: 0.3170
Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

/Users/ankur/backup/delta/sn/datascience/workspace/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8015 - loss: 0.4458
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8541 - loss: 0.3635
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8639 - loss: 0.3417
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8641 - loss: 0.3350
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8645 - loss: 0.3309
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8680 - loss: 0.3256
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8703 - loss: 0.3206
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.3180
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8714 - loss: 0.3150
Epoch 10/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8721 - loss: 0.3111
Epoch 11/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8718 - loss: 0.3093
Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

/Users/ankur/backup/delta/sn/datascience/workspace/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7997 - loss: 0.4582
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8367 - loss: 0.3924
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8510 - loss: 0.3665
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8536 - loss: 0.3552
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8565 - loss: 0.3493
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8586 - loss: 0.3452
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8597 - loss: 0.3434
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8581 - loss: 0.3408
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8601 - loss: 0.3392
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8608 - loss: 0.3378
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8612 - loss: 0.3370
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy